 #**README**







---







## About the notebook



This notebook demonstrates how an agentic AI solution can be used to transition your passive, pre-configured AI workflows into an active, agentic AI system using multiple agents. <br>



*Learning Objectives:* To understand,

- Use multiple agents to chain and generate a strategic response

<br>



*Prerequisites:*

- Skills: Python 3+, Basics of AI workflow concepts, Basics of Agentic AI vocabulary
- Difficulty Level: Intermediate


<br>



*Quick Start:*<br>

1. Clone the Repository

```

git clone https://github.com/PradnyaSA/MyAIExperiments.git

cd hands-on-notebooks

```



2. Set-Up Access  

 - Refer to [Google Colab](https://colab.research.google.com/) to get started instantly, for *free* !



 - Get your access key to the [OpenAI](https://platform.openai.com/account/api-keys) API.

   <br>



3. To set up API keys in Colab: Go to the "🔑" icon on the left sidebar (Secrets).

 - Click "Add new secret".

 - For the name, use 'openai_api_key'.

 - For the value, paste your OpenAI API key.

 - Make sure "Notebook access" is enabled for this secret.

<br>


4. Run the notebook

 - Open the notebook in Google Colab.

 - Run each cell or run all.

<br>

You are all set!

---
<br>





## Topic: Agentic AI Example - Transform Your AI workflow into a Marketing Insights Generator

#### Exercise 1: Basic Implementation (~15 minutes)

Learn the fundamental setup:
- Build an Agent and define parameters to set the persona for each agent.
- add functions to build a LaTeX document if we want a well-formatted PDF.
- print campaign text for quick demo

Expected Output:

- Campaign produced using system prompts and instructions provided during setup.

<br>


Tasks:

- Start with a given example, then try different product ideas using the same system prompt instructions

<br>


Challenges:


- Design prompts for financial analysis

- Handle different document types

- Implement error handling and retries<br>

<br>


### Performance Benchmarks - Extra Credit
Try running using different models and compare results.

### Sample Results - Extra Credit
Convert the response into a PDF.

### Key Concepts

**Plain LLM ** → gives a generic summary.

**Agentic multi-Agent system ** → coordinates roles, fetches live data, and generates a strategic plan.

---

**1) Setup Stage**

Let's import right libraries to work with and set up the API Key so that LLMs can be called. You key can even call a paid subscription or your customer LLMs too. In this example we'll use Gemini as the LLM to keep things simple.

In [12]:
import os
import time
from google import genai
from google.genai import types
from google.colab import userdata

In [ ]:
!pip install google-genai

In [14]:
os.environ["GOOGLEAPI"] = userdata.get('google-api-key')

In [15]:
client = genai.Client(api_key=userdata.get('google-api-key'))

In [16]:
MODEL_NAME = 'gemini-2.5-flash'

In [17]:
#Import display and Markdown from IPython for formatted rendering of generated response
from IPython.display import Markdown, display

In [18]:
def printmd(string):
    display(Markdown(string))

**2) Tool Definition**

Here we are allowing Agents to access realtime information by accessing Google Search, as LLMs are typically frozen in time and don't have uptodate information. It is called grounding. You could ground it to your organization data too by using a RAG (Retrieval Augmentation Generation) architecture too.

In [19]:
# --- 2. TOOL DEFINITION: Google Search Grounding ---
# This tool allows the Research Agent to access real-time information.
# The structure must match the Gemini API specification for search grounding.
GOOGLE_SEARCH_TOOL = types.Tool(
    google_search={}
)

**3) Agent Class**

This step helps define the structure of agents and how we interact, tools used (in this case Google Search)...

In [20]:
# --- 3. AGENT CLASS: Encapsulating Persona and LLM Calls ---

class Agent:
    """A specialized agent powered by a Gemini model with a fixed system instruction."""

    def __init__(self, name: str, system_prompt: str, tools: list = None):
        """
        Initializes the agent with its persona and optional tools.
        :param name: The display name of the agent (e.g., "Research Agent").
        :param system_prompt: The instruction defining the agent's role and output format.
        :param tools: A list of tools (e.g., [GOOGLE_SEARCH_TOOL]) the agent can use.
        """
        self.name = name
        self.system_prompt = system_prompt
        self.tools = tools if tools is not None else []
        print(f"--- {self.name} Initialized ---")

    def run(self, prompt: str) -> str:
        """
        Executes the agent's task with the provided input prompt.
        :param prompt: The task or context to process.
        :return: The generated text output.
        """
        print(f"\n[{self.name}]: Executing task...")

        # Simple exponential backoff for robustness
        for attempt in range(3):
            try:
                response = client.models.generate_content(
                    model=MODEL_NAME,
                    contents=prompt,
                    config=types.GenerateContentConfig(
                        system_instruction=self.system_prompt,
                        tools=self.tools,
                    )
                )
                print(f"[{self.name}]: Task completed successfully.")

                # Check for grounded sources if the tool was used
                if self.tools:
                    metadata = response.candidates[0].grounding_metadata
                    # FIX: The 'web' attribute is gone. We now check for 'web_search_queries'.
                    if metadata and hasattr(metadata, 'web_search_queries') and metadata.web_search_queries:
                        print(f"  -> Used Google Search for queries: {', '.join(metadata.web_search_queries)}")

                return response.text

            except Exception as e:
                print(f"[{self.name}]: Attempt {attempt + 1} failed: {e}")
                if attempt < 2:
                    time.sleep(2 ** attempt)  # Wait for 1s, 2s, then give up
                else:
                    raise RuntimeError(f"[{self.name}] failed after multiple retries.") from e

**4) Agents Definition**
Next, we define our agents as follows:

**Agent 1 – Research Agent**

**Purpose:** Collect market insights and competitor data.
**Skills/Tools:** Web search API, PDF/CSV reader, or mock “MarketData.json.”
**Example Output:**
“Top 3 competitors: DressSmart, BowTie, and ReadySetGo.
Market trend: 12% YoY growth in connected fitness devices.
Key sentiment: consumers prefer eco-friendly, app-integrated products.”

**Agent 2 – Strategist Agent**

**Purpose:** Analyze insights and craft the marketing positioning.
**Uses:** The output from the Research Agent as input.
**Example Output:**
“Positioning: ‘Smart wardrobe organizer for busy lives.’
Differentiator: integrates smart dressing & professionalism.
Target segment: active urban professionals, age 21–50.”

**Agent 3 – Creative Agent**

**Purpose:** Generate campaign content and visual ideas.
**Tools: **Text-to-image model (optional), prompt templates.
**Example Output:**
“Tagline: ‘Dress to Success’
Sample social caption: ‘Your wardrobe organizer just got smarter. Sync your goals with every weather condition and activity.’
**Mockup prompt:** futuristic organizer, glowing hangers, closet background.”

In the cell below, you can see how Natural Language prompts define what an agent can do; this is a significant departure from static RPA flows.

In [21]:
# --- 4. AGENT DEFINITIONS ---

research_agent = Agent(
    name="Research Agent",
    system_prompt=(
        "You are a professional Market Research Analyst. Your job is to collect and summarize "
        "market insights, trends, and competitor data. Your output MUST be concise, accurate, "
        "and formatted as three distinct bullet points covering: Top 3 Competitors, Market Trend, "
        "and Key Consumer Sentiment."
    ),
    tools=[GOOGLE_SEARCH_TOOL] # Provide web access for real-time data
)

strategist_agent = Agent(
    name="Strategist Agent",
    system_prompt=(
        "You are a Senior Marketing Strategist. Your job is to analyze the raw research data "
        "provided by the Research Agent and craft a clear product positioning. Your output MUST be "
        "formatted as three distinct bullet points covering: Positioning Statement, Differentiator, "
        "and Primary Target Segment."
    )
    # No tools needed; relies purely on the research input
)

creative_agent = Agent(
    name="Creative Agent",
    system_prompt=(
        "You are a world-class Creative Director. Your job is to take the positioning strategy "
        "and generate compelling campaign content. Your output MUST be formatted as three distinct "
        "bullet points covering: A catchy Tagline, a Sample Social Caption, and a detailed Mockup Prompt "
        "for an image generation model."
    )
    # No tools needed; relies purely on the strategy input
)

# --- NEW FUNCTION: LATEX GENERATION ---

def generate_latex_brief(product_description: str, market_insights: str, campaign_strategy: str, campaign_content: str) -> str:
    """
    Generates a complete LaTeX document string for the Final Campaign Brief.
    Includes placeholders for images as external files are not supported.
    FIX: Uses string addition (+=) to avoid SyntaxError with multi-line raw strings.
    FIX: Adds robust cleaning to handle messy agent output.
    """
    product_title = product_description.replace("\n", " ").strip()

    def clean_agent_output(text):
        """Aggressively cleans agent output to be safe for LaTeX itemization."""

        # 1. Clean up excessive whitespace and remove introductory sentences
        lines = text.strip().split('\n')
        content_lines = []
        # Start capturing content only after the first line that looks like a list item
        capture_started = False
        for line in lines:
            line_stripped = line.strip()
            if line_stripped.startswith(('*', '-')):
                capture_started = True

            if capture_started:
                # Remove Markdown and ensure a single bullet point structure
                cleaned_line = line_stripped.replace('**', '').replace('*', '').replace('-', '')

                # Check if the line is now just empty or a title after cleaning
                if cleaned_line.strip():
                    content_lines.append(cleaned_line)

        # Re-join and ensure every line starts with \item
        cleaned = '\n'.join(content_lines)

        # Replace the common agent output patterns ('\n\n') and markdown bullets
        cleaned = cleaned.replace('\n\n', '\n').replace('\n\n', '\n')
        cleaned = cleaned.replace('\n', r'\item ')

        # Ensure the string starts with \item and remove leading whitespace/text
        if cleaned.startswith(r'\item '):
            return cleaned
        else:
            return r'\item ' + cleaned

    # Start document and preamble
    latex_document = r"""\documentclass[11pt, a4paper]{article}
\usepackage[a4paper, top=2.5cm, bottom=2.5cm, left=2cm, right=2cm]{geometry}
\usepackage{fontspec}
\usepackage[english, bidi=basic, provide=*]{babel}
\babelfont{rm}{Noto Sans}
\usepackage{booktabs}
\usepackage{enumitem}
\usepackage{hyperref}

\title{\textbf{Campaign Brief: \MakeUppercase{"""

    # 1. Insert product_title
    latex_document += product_title.upper()

    # Continue Document (Title end, Author, Product Description start)
    latex_document += r"""}}}
\author{Agentic Workflow (Research, Strategy, Creative)}
\date{\today}

\begin{document}
\maketitle
\thispagestyle{empty} % No page number on the title page

\vspace{1em}
\hrule
\vspace{1em}

\section*{Product Overview}
\noindent \textbf{Description:} """

    # 2. Insert product_description
    latex_document += product_description

    # Continue Document (Market Insights start)
    latex_document += r"""

\section*{A. Market Insights}
\subsection*{Data from Research Agent}
\begin{itemize}[label=$\bullet$]
"""

    # 3. Insert market_insights (with cleanup)
    latex_document += clean_agent_output(market_insights)

    # Continue Document (Strategy start)
    latex_document += r"""
\end{itemize}

% Placeholder for a catchy image (Market Trend)
\begin{figure}[htbp]
  \centering
  \framebox{\parbox{0.8\textwidth}{\centering
    \vspace{2cm}
    \textbf{Visualizing Market Growth Trend} \\
    \small\textit{Placeholder for a competitor analysis chart or growth graph.}
    \vspace{2cm}
  }}
  \caption{Market Trend Visual Placeholder.}
  \label{fig:market_trend}
</figure>

\section*{B. Strategy \& Positioning}
\subsection*{Analysis from Strategist Agent}
\begin{itemize}[label=$\bullet$]
"""

    # 4. Insert campaign_strategy (with cleanup)
    latex_document += clean_agent_output(campaign_strategy)

    # Continue Document (Creative Content start)
    latex_document += r"""
\end{itemize}

\section*{C. Creative Content}
\subsection*{Output from Creative Agent}
\begin{itemize}[label=$\bullet$]
"""

    # 5. Insert campaign_content (with cleanup)
    latex_document += clean_agent_output(campaign_content)

    # End Document
    latex_document += r"""
\end{itemize}

% Placeholder for a catchy image (Creative Mockup)
\begin{figure}[htbp]
  \centering
  \framebox{\parbox{0.8\textwidth}{\centering
    \vspace{3cm}
    \textbf{Campaign Mockup Visual} \\
    \small\textit{Placeholder for the futuristic wardrobe organizer ad based on the mockup prompt.}
    \vspace{3cm}
  }}
  \caption{Creative Ad Mockup Placeholder.}
  \label{fig:ad_mockup}
</figure>

\vspace{2em}
\hrule
\begin{center}
\textbf{END OF CAMPAIGN BRIEF}
\end{center}
\hrule
\end{document}
"""
    return latex_document

# --- 5. MANAGER AGENT LOGIC (The Workflow/Chaining Function) ---

def run_campaign_deck_workflow(product_description: str):
    """
    Coordinates the three specialized agents sequentially to produce a campaign deck
    and generate the final LaTeX document string.
    """
    print("\n\n#################################################")
    print(f"STARTING CAMPAIGN WORKFLOW for: {product_description}")
    print("#################################################")

    # Step 1: Research Agent (Requires grounding tool)
    research_prompt = (
        f"Research the market for a product described as '{product_description}'. "
        "I need a summary of the top 3 competitors, the latest market trend, and key consumer sentiment."
    )
    research_agent.system_prompt += " Ensure your bullet points are simple, single lines, and do not contain introductory sentences or nested lists."
    market_insights = research_agent.run(research_prompt)
    print("\n--- RESEARCH AGENT OUTPUT ---\n" + market_insights)

    # Step 2: Strategist Agent (Uses research output as input)
    strategy_prompt = (
        "Based on the following market insights, define the core marketing positioning, "
        "the key differentiator, and the primary target segment for the new product:\n\n"
        f"MARKET INSIGHTS:\n{market_insights}"
    )
    campaign_strategy = strategist_agent.run(strategy_prompt)
    print("\n--- STRATEGIST AGENT OUTPUT ---\n" + campaign_strategy)

    # Step 3: Creative Agent (Uses strategy output as input)
    creative_prompt = (
        "Using the campaign strategy below, generate a catchy tagline, a sample social "
        "media caption, and a detailed text-to-image mockup prompt for a visual ad:\n\n"
        f"CAMPAIGN STRATEGY:\n{campaign_strategy}"
    )
    campaign_content = creative_agent.run(creative_prompt)
    print("\n--- CREATIVE AGENT OUTPUT ---\n" + campaign_content)

    # Step 4: Compile Final Brief (Text Version)
    final_brief = (
        "\n\n================================================="
        "\n            FINAL CAMPAIGN BRIEF (Text)"
        "\n================================================="
        f"\n\n**PRODUCT:** {product_description}"
        f"\n\n**A. MARKET INSIGHTS**\n{market_insights}"
        f"\n\n**B. STRATEGY & POSITIONING**\n{campaign_strategy}"
        f"\n\n**C. CREATIVE CONTENT**\n{campaign_content}"
        "\n================================================="
    )
    print(final_brief)

    # Step 5: Generate LaTeX (PDF) version
    latex_output = generate_latex_brief(
        product_description,
        market_insights,
        campaign_strategy,
        campaign_content
    )

    print("\n\n#################################################")
    print("LATEX SOURCE CODE GENERATED (for PDF creation)")
    print("#################################################")
    # Print the raw LaTeX output so the user can copy/save it as a .tex file
    print(latex_output)

    return final_brief, latex_output

--- Research Agent Initialized ---
--- Strategist Agent Initialized ---
--- Creative Agent Initialized ---


**6) Execution: Running the Agents once tied together**

**🧠 Coordination Fl**ow

**Manager Agent (optional):** orchestrates steps sequentially:

Ask Research Agent for data

Pass data to Strategist Agent

Send strategy to Creative Agent for content

Compile final brief and output a cohesive campaign deck


In [ ]:
# --- 6. EXECUTION ---
# Define the product the agents will work on
PRODUCT_TO_LAUNCH = "An AI-powered Smart Wardrobe Organizer that dynamically adjusts wardrobe combinations based on local weather and user activity."

# Run the full agentic flow
final_brief, latex_source = run_campaign_deck_workflow(PRODUCT_TO_LAUNCH)

In [23]:
about_this_print='Following is how the agentic AI system responds with multi-agent set up:'
printmd('<div style="background-color: lightblue; padding: 10px;">%s</div><br>' % about_this_print)
printmd('<div style="background-color: lightblue; padding: 10px;">%s</div>' % final_brief)

<div style="background-color: lightblue; padding: 10px;">Following is how the agentic AI system responds with multi-agent set up:</div><br>

<div style="background-color: lightblue; padding: 10px;">

=================================================
            FINAL CAMPAIGN BRIEF (Text)
=================================================

**PRODUCT:** An AI-powered Smart Wardrobe Organizer that dynamically adjusts wardrobe combinations based on local weather and user activity.

**A. MARKET INSIGHTS**
Here's a summary of the market for an AI-powered Smart Wardrobe Organizer:

*   **Top 3 Competitors:** Alta (AI outfit planning, style analytics, weather/events integration), Whering (AI-driven outfit suggestions, weather consideration, digital wardrobe management), and Acloset (AI-based outfit combinations, daily planning, weather-appropriate suggestions).
*   **Market Trend:** The smart wardrobe market is experiencing rapid expansion with a strong emphasis on AI-driven personalization, seamless integration with smart home ecosystems, and increasing demand for sustainable fashion solutions.
*   **Key Consumer Sentiment:** Consumers are increasingly trusting and interested in AI for personalized outfit recommendations and efficient wardrobe management, though data privacy and the desire for accurate, context-aware suggestions (including weather and activity) remain crucial.

**B. STRATEGY & POSITIONING**
Here's the core marketing positioning for the new AI-powered Smart Wardrobe Organizer:

*   **Positioning Statement:** For the contemporary individual seeking effortless style and efficient living, our AI-powered Smart Wardrobe Organizer delivers **hyper-personalized outfit recommendations and holistic wardrobe management**, transforming daily dressing into a seamless and confident experience.
*   **Differentiator:** Our key differentiator lies in its **seamless integration with broader smart home ecosystems**, its **empowerment of sustainable fashion choices through data-driven insights**, and its **unwavering commitment to user data privacy** that builds trust.
*   **Primary Target Segment:** Fashion-conscious and tech-savvy individuals who actively seek intelligent solutions to streamline their daily routines, value personalized experiences, and are increasingly concerned with making sustainable lifestyle choices.

**C. CREATIVE CONTENT**
Here's the campaign content for the AI-powered Smart Wardrobe Organizer:

*   **Catchy Tagline:** Your Style, Amplified. Intelligently. Sustainably.

*   **Sample Social Caption:**
    Tired of outfit indecision? Meet your new AI-powered stylist! ✨ Our Smart Wardrobe Organizer isn't just about clothes – it's about transforming your daily routine. Get hyper-personalized outfit recommendations, effortlessly discover new looks, and make sustainable fashion choices, all while seamlessly integrating with your smart home. We believe in intelligent style, ethical choices, and your absolute data privacy. Ready to step out with confidence, every single day? Link in bio to learn more! #SmartStyle #AIPowered #SustainableFashion #EffortlessChic #SmartHomeIntegration #FutureOfFashion

*   **Mockup Prompt for Image Generation Model:**
    High-resolution, cinematic photograph: A confident, stylish woman (30s, diverse ethnicity, radiating modern elegance) stands in a meticulously organized, minimalist walk-in closet. She wears a perfectly curated, chic business casual outfit (e.g., tailored wide-leg trousers, a silk blouse, minimalist jewelry). Her hand lightly touches a large, transparent smart screen seamlessly integrated into the wardrobe wall. The screen displays a glowing, holographic interface with her chosen outfit, AI-generated accessory suggestions, and data-driven insights on sustainable fabric alternatives. Subtle smart home integration icons are visible on the screen's periphery. The wardrobe's open shelves and hanging racks are organized with high-quality, diverse clothing, subtly lit by futuristic LED strips that cast a soft glow. Soft, diffused natural morning light floods the scene from a large window, enhancing textures and creating a serene glow. The mood is sophisticated, empowering, and effortlessly efficient. Focus on clean lines, aspirational aesthetic, and a sense of calm confidence. Color palette: muted neutrals (grays, creams, whites) with sophisticated pops of color in the clothing, and cool blue/green hues for the holographic interface.
=================================================</div>

---

##Congratulations on running this fun exercise!
This lab is designed to provide you with practical, hands-on experience with an agentic AI system that utilizes multiple agents. Take your time with the exercise and don't hesitate to experiment beyond the provided example.


Happy Learning!